In [ ]:
#run this cell and then restart execution
!pip install accelerate
!pip install bitsandbytes
!pip install optimum
!pip install auto-gptq
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install -U gradio==3.32.0

In [1]:
import gradio as gr

In [2]:
!pip install transformers
from transformers import AutoModelForCausalLM,AutoTokenizer
import torch
!pip install transformers huggingface_hub
#from huggingface_hub import notebook_login

#notebook_login()


In [3]:
mn = 'stabilityai/StableBeluga-7B'
#mn = "TheBloke/Llama-2-7b-Chat-GPTQ"

In [4]:
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, load_in_8bit=True)

#model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.float16)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
sb_sys = "### System:\nYou are a AI assistant in my car, that follows instructions extremely well. Help as much as you can. Answer questions concisely and effectively with the requested information.\n\n"


In [6]:
def gen(p, maxlen=15, sample=True):
    toks = tokr(p, return_tensors="pt")
    res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample).to('cpu')
    return tokr.batch_decode(res)

In [7]:
tokr = AutoTokenizer.from_pretrained(mn)

In [8]:
#to have a prompt corresponding to the specific format required by the fine-tuned model Stable Beluga
def mk_prompt(user, syst=sb_sys): return f"{syst}### User: {user}\n\n### Assistant:\n"

In [9]:
complete_answer= ''

In [10]:
# get user location

import requests

response = requests.get("http://ip-api.com/json/")
data = response.json()
print(data['city'], data['lat'], data['lon'])
city= data['city']
lat = data['lat']
lon = data['lon']


Taipei 25.0329 121.5654


In [11]:
#weather API

import requests

def get_weather_with_weatherapi(api_key:str, city_name:str):
    # The endpoint URL provided by WeatherAPI
    url = f"http://api.weatherapi.com/v1/current.json?key={api_key}&q={city_name}&aqi=no"

    # Make the API request
    response = requests.get(url)

    if response.status_code == 200:
        # Parse the JSON response
        weather_data = response.json()

        # Extracting the necessary pieces of data
        location = weather_data['location']['name']
        region = weather_data['location']['region']
        country = weather_data['location']['country']
        time = weather_data['location']['localtime']
        temperature_c = weather_data['current']['temp_c']
        condition_text = weather_data['current']['condition']['text']
        wind_mph = weather_data['current']['wind_mph']
        humidity = weather_data['current']['humidity']
        feelslike_c = weather_data['current']['feelslike_c']

        # Formulate the sentences
        weather_sentences = (
            f"The current weather in {location}, {region}, {country} is {condition_text} "
            f"with a temperature of {temperature_c}°C that feels like {feelslike_c}°C. "
            f"Humidity is at {humidity}%. "
            f"Wind speed is {wind_mph} mph."
        )
        return weather_sentences
    else:
        # Handle errors
        return f"Failed to get weather data: {response.status_code}, {response.text}"

# Replace 'YOUR_API_KEY' with your actual API key from WeatherAPI and 'CITY_NAME' with your city
api_key = 'a08df3f6b6264a1facc135137230311'
city_name = city
weather_info = get_weather_with_weatherapi(api_key, city_name)
print(weather_info)


The current weather in Taipei, T'ai-pei, Taiwan is Partly cloudy with a temperature of 19.0°C that feels like 19.0°C. Humidity is at 83%. Wind speed is 8.1 mph.


In [12]:
import requests
import json

results = []
r= requests.get('https://api.tomtom.com/search/2/poiCategories.json?key=FY0suJKC0xfktJmHjnNOGLXRmSyvS6zi')
for result in r.json()['poiCategories']:
    results.append(f"id : {result['id']}, name: {result['name']}")
print (results) #categories number, to use when llama2 will be able to call functions &categorySet=number

tomtom_key = "FY0suJKC0xfktJmHjnNOGLXRmSyvS6zi"
def find_points_of_interest(lat, lon, city, key, type_of_poi):
    """
    :param lat: latitude
    :param lon: longitude
    :param key: api key
    :param type: type of poi
    :return: [5] results ['poi']['name']/['freeformAddress'] || ['position']['lat']/['lon']
    """
    results = []

    r = requests.get(f'https://api.tomtom.com/search/2/search/{type_of_poi}'
                     '.json?key={0}&lat={1}&lon={2}&radius=10000&idxSet=POI&limit=100'.format(
                        key,
                        lat,
                        lon
    ))
    for result in r.json()['results']:
        results.append(f"The {type_of_poi} {result['poi']['name']} is {int(result['dist'])} meters from {city}")
        if len(results) == 7:
            break

    return ". ".join(results)


print(find_points_of_interest('49.625892805337514', '6.160417066963513', 'your location', tomtom_key, 'sushi'))

['id : 7320, name: Sports Center', 'id : 7374, name: Stadium', 'id : 7315, name: Restaurant', 'id : 9376, name: Café/Pub', 'id : 9663, name: Health Care Service', 'id : 7321, name: Hospital', 'id : 9373, name: Doctor', 'id : 9361, name: Shop', 'id : 9364, name: Marijuana Dispensary', 'id : 7332, name: Market', 'id : 9932, name: Public Amenity', 'id : 7301, name: Road Traffic Control Center', 'id : 7324, name: Post Office', 'id : 7377, name: College/University', 'id : 7369, name: Open Parking Area', 'id : 9937, name: Club & Association', 'id : 7360, name: Campground', 'id : 7304, name: Vacation Rental', 'id : 9902, name: Amusement Park', 'id : 7303, name: Residential Accommodations', 'id : 7335, name: Agricultural Business', 'id : 7383, name: Airport', 'id : 9352, name: Company', 'id : 9379, name: Nightlife', 'id : 7310, name: Repair Shop', 'id : 7314, name: Hotel/Motel', 'id : 7372, name: School', 'id : 9910, name: Automotive Dealer', 'id : 9927, name: Zoo, Arboretum & Botanical Garden

In [21]:
import re
api_key_weather= 'a08df3f6b6264a1facc135137230311'
tomtom_key = "FY0suJKC0xfktJmHjnNOGLXRmSyvS6zi"
list_poi = ['restaurant', 'hotel', 'café', 'bank', 'theater', 'pizza', 'sushi']
cities = [('Kirchberg', '49.625892805337514', '6.160417066963513'), (city, lat, lon), ('Esch-sur-Alzette', '49.4959628', '5.9850306')]


pois = ". ".join(find_points_of_interest( lat, lon, city, tomtom_key, poi)
                for poi in list_poi)
#for city in cities

weathers_in_various_locations = [get_weather_with_weatherapi(api_key_weather, 'Luxembourg'), get_weather_with_weatherapi(api_key_weather, city), get_weather_with_weatherapi(api_key_weather, 'Esch-sur-Alzette')]
model_answer= ''
general_context= f'I am in my car in {city}. My exact position is latitude {lat}, longitude {lon}. I can move with my car to reach a destination. {". ".join(weathers_in_various_locations)} {pois}'
# Define the initial state with some initial context.
print(general_context)
initial_state = {'context': general_context}
pattern = r"Assistant:\\n(.*?)</s>"
#The api key for weather (and only for weather) is a08df3f6b6264a1facc135137230311

I am in my car in Taipei. My exact position is latitude 25.0329, longitude 121.5654. I can move with my car to reach a destination. The current weather in Luxembourg, Luxembourg, Luxembourg is Patchy light rain with a temperature of 3.0°C that feels like -2.4°C. Humidity is at 93%. Wind speed is 15.0 mph.. The current weather in Taipei, T'ai-pei, Taiwan is Partly cloudy with a temperature of 19.0°C that feels like 19.0°C. Humidity is at 83%. Wind speed is 8.1 mph.. The current weather in Esch-Sur-Alzette, Luxembourg, Luxembourg is Patchy light rain with a temperature of 3.0°C that feels like -2.3°C. Humidity is at 93%. Wind speed is 15.0 mph. The restaurant Brasserie du Kirchberg is 1043 meters from Kirchberg. The restaurant Piri Piri Portuguese Restaurant & Bar is 118 meters from Kirchberg. The restaurant Häagen-Dazs Galerie Du Kirchberg is 1042 meters from Kirchberg. The restaurant Vida Sushi Lounge-Kirchberg is 262 meters from Kirchberg. The restaurant Superfoods is 489 meters from 

In [16]:
def car_answer_only(complete_answer, general_context):
    match = re.search(pattern, complete_answer, re.DOTALL)

    if match:
        # Extracting the text
        model_answer = match.group(1)
    else:
        model_answer = "There has been an error with the generated response."

    general_context +=  model_answer
    return (model_answer, general_context)
#print(model_answer)

In [17]:
def FnAnswer(general_context, ques, state):
    # Initialize state if it is None
    if state is None:
        state = {}
        state['context'] = general_context


    question=f"""Answer the question with the help of the provided context.

    ## Context

    {general_context} .

    ## Question

    {ques}"""

    complete_answer = str(gen(mk_prompt(question), 300))

    model_answer, general_context= car_answer_only(complete_answer, general_context)

    state['context'] = state['context'] + ". " + model_answer


    return model_answer, state['context'], state

In [ ]:
initial_context= initial_state['context']
# Create the Gradio interface.
iface = gr.Interface(
    fn=FnAnswer,
    inputs=[
        gr.Textbox(value=initial_context, visible=False),
        gr.Textbox(lines=2, placeholder="Type your message here..."),
        gr.State()  # This will keep track of the context state across interactions.
    ],
    outputs=[
        gr.Textbox(),
        gr.Textbox(visible=False),
        gr.State()
    ]
)
# Launch the interface.
iface.launch(debug=True, share=True)
#contextual=gr.Textbox(value=general_context, visible=False)
#demo = gr.Interface(fn=FnAnswer, inputs=[contextual,"text"], outputs=["text", contextual])

#demo.launch()

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://ba29e1c6314b2a9318.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/routes.py", line 422, in run_predict
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1323, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1051, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 807, in run
    result = context.run(func, *args)
  File "<ipython-input-17-92dcc911dd46>", line 18, in FnAnswer
    complete_answer = str(gen(mk_prompt(question), 300))
  Fil

In [ ]:
#traffic API

import requests

api_key = 'YOUR_API_KEY'  # Replace with your actual TomTom API key
version_number = '1'  # Check the latest API version in the documentation

# TomTom Traffic API endpoint example for traffic incidents
traffic_incidents_endpoint = f"https://api.tomtom.com/traffic/services/5/incidentDetails?bbox=4.8854592519716675%2C52.36934334773164%2C4.897883244144765%2C52.37496348620152&fields=%7Bincidents%7Btype%2Cgeometry%7Btype%2Ccoordinates%7D%2Cproperties%7BiconCategory%7D%7D%7D&language=en-GB&categoryFilter=0%2C1%2C2%2C3%2C4%2C5%2C6%2C7%2C8%2C9%2C10%2C11%2C14&timeValidityFilter=present&key=FY0suJKC0xfktJmHjnNOGLXRmSyvS6zi"



response = requests.get(traffic_incidents_endpoint, params=params)

if response.status_code == 200:
    # Process the response if successful
    traffic_data = response.json()
    # Implement your logic to incorporate traffic_data into your language model
    print(traffic_data)
else:
    print(f"Error: {response.status_code}")

NameError: ignored

In [ ]:
#schema of function, may be useful to use python functions in the model (not sure)
from pydantic import create_model
import inspect, json
from inspect import Parameter



def schema(f):
    kw = {n:(o.annotation, ... if o.default==Parameter.empty else o.default)
          for n,o in inspect.signature(f).parameters.items()}
    s = create_model(f'Input for `{f.__name__}`', **kw).schema()
    return dict(name=f.__name__, description=f.__doc__, parameters=s)

In [ ]:
schema(get_weather_with_weatherapi)

{'name': 'get_weather_with_weatherapi',
 'description': None,
 'parameters': {'title': 'Input for `get_weather_with_weatherapi`',
  'type': 'object',
  'properties': {'api_key': {'title': 'Api Key', 'type': 'string'},
   'city_name': {'title': 'City Name', 'type': 'string'}},
  'required': ['api_key', 'city_name']}}